## Tensorflow Model 만들기

<데이콘 고객 대출등급 분류 헤커톤>

### 데이터 불러오기

In [1]:
from zipfile import ZipFile
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as f:
    data=io.BytesIO(f.read("고객 대출등급 분류 해커톤/train.csv"))

In [2]:
from pandas import read_csv
data=read_csv(data)

In [3]:
data.head(4)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C


### 전처리

tensorflow 의 Dataset API 에 대한 설명 이전이므로 table 형태로 사용하겠습니다.

In [4]:
# 표기된 nan 값은 없습니다
data.isna().sum()

ID              0
대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
대출등급            0
dtype: int64

In [5]:
data.대출기간.unique()

array([' 36 months', ' 60 months'], dtype=object)

근로기간 전처리

In [81]:
from sklearn import pipeline as pip
from sklearn import preprocessing as pre
from sklearn import compose as com
from sklearn import base as base
from sklearn import impute as imp
from sklearn import utils as utils
from sklearn import model_selection as mod

In [6]:
data.근로기간.unique()

array(['6 years', '10+ years', '5 years', '8 years', 'Unknown', '9 years',
       '2 years', '1 year', '3 years', '7 years', '4 years', '< 1 year',
       '10+years', '<1 year', '3', '1 years'], dtype=object)

In [45]:
import numpy as np
class Processing_Period(base.BaseEstimator,base.TransformerMixin):
    def __init__(self):
        super().__init__()
        self.__baseperiod__=[[
            "Unknown",
            "<1",
            "1","2","3","4","5","6","7","8","9",
            "10+"
        ]]
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        replace_X=[]
        for sub_x in X:
            if "years" in sub_x:
                sub_=sub_x.replace("years","").replace(" ","").strip()
            elif "year" in sub_x:
                sub_=sub_x.replace("year","").replace(" ","").strip()
            else:
                sub_=sub_x.replace(" ","").strip()
            replace_X.append(sub_)
        return np.array(replace_X).reshape(-1,1)

baseperiod__=[[
            "Unknown",
            "<1",
            "1","2","3","4","5","6","7","8","9",
            "10+"]]
period_text_=pre.OrdinalEncoder(categories=baseperiod__)
period_preprocessing=pip.Pipeline(
    [("text_preprocessing",Processing_Period()),
     ("ordinary_preprocessing",period_text_)]
)

In [47]:
period_preprocessing.fit_transform(data.근로기간.tolist())

array([[ 7.],
       [11.],
       [ 6.],
       ...,
       [ 2.],
       [ 6.],
       [11.]])

대출기간

두가지 값밖에 없어서 OneHotEncoder를 사용하겠습니다.

In [54]:
data.대출기간.unique() 

array([' 36 months', ' 60 months'], dtype=object)

In [56]:
data.주택소유상태.unique()

array(['RENT', 'MORTGAGE', 'OWN', 'ANY'], dtype=object)

In [58]:
data.대출목적.unique()

array(['부채 통합', '주택 개선', '주요 구매', '휴가', '의료', '자동차', '신용 카드', '소규모 사업',
       '기타', '이사', '주택', '재생 에너지'], dtype=object)

In [61]:
onehotencoder=pre.OneHotEncoder(categories=[[' 36 months', ' 60 months'],
                              ['RENT', 'MORTGAGE', 'OWN', 'ANY'],
                              ['부채 통합', '주택 개선', '주요 구매', '휴가', '의료', '자동차', '신용 카드', '소규모 사업',
       '기타', '이사', '주택', '재생 에너지']],handle_unknown="ignore",drop="if_binary")

#### 데이터 전처리를 마무리

In [69]:
# 7개의 대출등급이 있네요.
label_cates=data.대출등급.unique()
label_cates.sort()
print(label_cates)

In [80]:
# 타겟도 다음과 같이 처리합니다.
# 일반적으로 10개 이상일 경우 ordinary로 처리합니다. 
label_encoder=pre.LabelBinarizer() # softmax 형태

label_encoder=pre.LabelEncoder() # 순서대로 처리

In [78]:
label_encoder.fit(data.대출등급)
print("학습된 label 종류와 순서 :",label_encoder.classes_)

학습된 label 종류와 순서 : ['A' 'B' 'C' 'D' 'E' 'F' 'G']


In [84]:
num_cols=data.select_dtypes(np.number).columns

In [86]:
data.select_dtypes("object").columns

Index(['ID', '대출기간', '근로기간', '주택소유상태', '대출목적', '대출등급'], dtype='object')

In [121]:
features_pipeline=com.ColumnTransformer(
    [("numeric",pre.StandardScaler(),num_cols),
     ("근로기간",period_preprocessing,"근로기간"),
     ("cates",onehotencoder,['대출기간','주택소유상태','대출목적'])],remainder="drop")

In [122]:
label="대출등급"
features=data.columns.difference([label])

변환

In [130]:
use_X,use_y=features_pipeline.fit_transform(X=data[features]),label_encoder.fit_transform(data[label])

In [148]:
train_X,test_X,train_y,test_y=mod.train_test_split(use_X,use_y,train_size=0.8,random_state=1,stratify=use_y)
valid_X,test_X,valid_y,test_y=mod.train_test_split(test_X,test_y,train_size=0.8,random_state=1,stratify=test_y)

### Keras Model

케라스 모델링은 3가지 방법을 제시하고 있습니다.

1. sequential - 간단함
2. funtional - 복잡함 추가 + 유연성
3. subclassing - 더 다양한 유연성 + (keras가 쉽게 검사하기 어려움)

하나하나 따라해봅시다

#### 1. Sequntial

> 굉장히 간단하고 사용하기 편합니다

In [151]:
import keras
sequentail_model=keras.Sequential(
    [keras.layers.Dense(units=64,activation="relu"),
     keras.layers.Dense(units=32,activation="relu"),
     keras.layers.Dense(units=1,activation="relu")]
)

/Users/ijeonghun/Documents/github/DL/DL/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [152]:
sequentail_model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])

2024-12-30 17:23:11.707894: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-12-30 17:23:11.707916: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-12-30 17:23:11.707922: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-12-30 17:23:11.708111: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-30 17:23:11.708125: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [154]:
sequentail_model.fit(x=train_X,y=train_y,epochs=3,batch_size=10,validation_data=(valid_X,valid_y),validation_batch_size=10)

Epoch 1/3
7704/7704 ━━━━━━━━━━━━━━━━━━━━ 68s 9ms/step - accuracy: 0.1762 - loss: 0.0000e+00 - val_accuracy: 0.1741 - val_loss: 0.0000e+00
Epoch 2/3
7704/7704 ━━━━━━━━━━━━━━━━━━━━ 65s 8ms/step - accuracy: 0.1743 - loss: 0.0000e+00 - val_accuracy: 0.1741 - val_loss: 0.0000e+00
Epoch 3/3
7704/7704 ━━━━━━━━━━━━━━━━━━━━ 66s 9ms/step - accuracy: 0.1741 - loss: 0.0000e+00 - val_accuracy: 0.1741 - val_loss: 0.0000e+00


예시이므로 성능에 관련된 추가적인 설정은 미루겠습니다.

- 파이썬 문법 적용하기 

> for, while, if 문 등등

In [157]:
sequentail_model_2=keras.Sequential([])
for layer_index in range(3):
    if layer_index<=2:
        sequentail_model_2.add(
            keras.layers.Dense(64,activation="relu")
        )
    else:
        sequentail_model_2.add(
            keras.layers.Dense(1,activation="relu")
        )

#### 2. funtional

> Sequential 과 다른 input layer의 정의가 필요합니다

In [170]:
# None으로 정의하면 들어오는 데이터의 형태를 통해 정합니다.
input_layer=keras.layers.Input(shape=use_X.shape[1:])

In [171]:
hidden_layer_1=keras.layers.Dense(64,activation="relu")(input_layer)
hidden_layer_2=keras.layers.Dense(64,activation="relu")(hidden_layer_1)
output_layer=keras.layers.Dense(1,activation="relu")(hidden_layer_2)

In [172]:
functional_model=keras.Model(inputs=input_layer,outputs=output_layer)

#### 3. subclassing

위의 예시를 직접 해본다면 `hidden_layer_1`에서 문제가 생김을 알수 있습니다.

즉 데이터의 기본적인 구조를 요구하는 구조라는 뜻입니다.

이를 해결하기 위해선 더 강한 유연성이 필요합니다.

In [177]:
class SubclassingModel(keras.Model):
    def __init__(self,units:int,activation:str="relu",*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden_layer_1=keras.layers.Dense(units,activation=activation)
        self.hidden_layer_2=keras.layers.Dense(units,activation=activation)
        self.output_layer=keras.layers.Dense(1,activation=activation)

    def call(self,inputs):
        hidden_1=self.hidden_layer_1(inputs)
        hidden_2=self.hidden_layer_1(hidden_1)
        output=self.output_layer(hidden_2)
        return output

In [178]:
subclass_model=SubclassingModel(units=64,activation="relu")

위의 예시처럼 서브클래싱 방법은 `Input`객체를 생성하지 않고 `call`메서드를 통해 inputs 를 호출합니다.

#### 주의사항

`Keras.Model` 은 `input`과 `oupput` 속성을 가지고 있어서 변수명이 겹치지않게 조심하여야합니다

### 모델의 저장과 불러오기

앞으로 우리는 여러 프로젝트를 진행하면서 한번에 완벽한 모델을 완성할 수 없습니다.

따라서 keras에서 생성된 모델을 저장하고 불러오는 법을 알아보겠습니다.

> keras3 를 사용중이면 (keras 단독으로)핸즈온에서 설명하는 방법으로 저장할 수 없습니다. 확장자명은 `.keras` ,`.h5` 둘중 하나를 사용해야합니다

케라스 버전

In [187]:
sequentail_model.save("my_sequential_model.keras")

텐서플로우의 `SaveModel` 포멧을 사용하기 위해선 아래의 방법을 사용합니다

In [193]:
import tensorflow as tf
tf.saved_model.save(sequentail_model,"my_sequential_model")

INFO:tensorflow:Assets written to: my_sequential_model/assets


INFO:tensorflow:Assets written to: my_sequential_model/assets


후자의 방법이 `tensorflow`에서 요구하는 포멧입니다.

또한 `asset`디렉토리에 변수명 중 다양한 정보를 저장해놓을 수 있습니다(기본적으로 빈 디렉토리입니다)